In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import torch
import matplotlib.pyplot as plt
import os
import glob
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, jaccard_score


In [7]:
img_list = glob.glob('D:test//input//*.png')
GMM_list = glob.glob('D:test//output//*.png')
DL_list = glob.glob('D:pred_test//*.png')
all_DL = glob.glob('D:all_pred//*.png')

In [8]:
# Dice similarity function
def dice(true, pred, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice

In [10]:
# all the segmentation metrics
def get_metrics (y_true, y_pred):
    percision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1_sc = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    AoU = jaccard_score(y_true, y_pred)
    Dice = dice(y_true, y_pred)
    pixels = len(y_pred[y_pred>0])
   
    
    return percision, recall,f1_sc ,accuracy, AoU, Dice, pixels
    

In [12]:
# getting number of ice pixels in the prediction set of deep learning model to calculate the predicted weight by DL model
N_elements = len(all_DL)
pixels = np.zeros(N_elements)
names = []

for i in range(N_elements):
    y_pred = cv2.imread(all_DL[i], cv2.IMREAD_GRAYSCALE).reshape(-1)
    pixels[i] = len(y_pred[y_pred > 0])
    names.append(all_DL[i].split('\\')[-1])
    print (round( 100*(i+1)/len(all_DL), 1),  end="\r")

In [13]:
# chaning the numpy array to a pandas dataframe to have an excel output of pixels for calculating the weights
data = pd.DataFrame(pixels, columns=[ 'pixels'])
data ['names'] = names
data.head()
data.to_excel('all_weights.xlsx', index=False)

In [5]:
# calculating the model metrics and putting in a numpy array along with the image names
metrics = np.zeros((len(img_list),7))
names = []

for i in range(len (img_list)):
    y_true = cv2.imread(GMM_list[i], cv2.IMREAD_GRAYSCALE).reshape(-1)
    #binarization of ground truth
    y_true = y_true / y_true.max()
    y_true [y_true> 0.5] = 1
    y_true [y_true <= 0.5] = 0

    y_pred = cv2.imread(DL_list[i], cv2.IMREAD_GRAYSCALE).reshape(-1)
    
    _m = get_metrics(y_true, y_pred)
    
    metrics[i] = _m
    names.append(img_list[i].split('\\')[-1])
    print (round( 100*(i+1)/len(img_list), 1),  end="\r")

In [30]:
# gathering model metrics from the numpy array and adding the image name and saving as a excel file
data = pd.DataFrame(metrics, columns=['percision', 'recall','f1_score' ,'accuracy', 'AoU', 'Dice', 'pixels'])
data ['names'] = names
data.head()
data.to_excel('metrics_3.xlsx', index=False)

,percision,recall,f1_score,accuracy,AoU,Dice,pixels
0,0.979134,0.975343,0.977235,0.990641,0.955483,0.977235,860522.0
1,0.976776,0.977734,0.977255,0.990616,0.955521,0.977255,865639.0
2,0.975796,0.978722,0.977257,0.990587,0.955525,0.977257,869277.0
3,0.975134,0.979036,0.977081,0.990494,0.955190,0.977081,871601.0
4,0.980127,0.974399,0.977255,0.990597,0.955521,0.977255,864405.0
...,...,...,...,...,...,...,...
1592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1594,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
